In [1]:
# Create the proper folder structure for the TCGA dataset. This script assumes the following:
#  TCGA slides are loaded into a collection
#  Under the collection are the cohorts folders (ACC, LGG, GBM, etc.)
#  Under each cohort are the list of slides

# The create folder structure will be as follows:
# collection -> cohorts -> cases (patient folders) -> slide folders

In [1]:
#URL to the API
BASE_URL = 'http://gauss.neurology.emory.edu:8080/api/v1'

#TCGA collection name
COLLECTION_NAME = 'TCGA'

In [5]:
import json
import girder_client
import collections
import sys
sys.path.append("../utils")
from girder_utils import *

In [8]:
gc = girder_client.GirderClient(apiUrl=BASE_URL)
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [9]:
##Going to get my collection by ID so I can interact with it
requestUrl = 'resource/lookup?path=collection/%s' % COLLECTION_NAME
collectionData = gc.getResource(requestUrl)
collectionId = collectionData['_id']

In [ ]:
#First, for every cohort (main folder) do the following
#  1. Get list of items and subfolders
#  2. Map folder name to folder ID
#  3. Delete any item with filename starting with .TCGA
#  4. Create a folder for each patient, of the folder does not exist
folders = recurseGetResource(gc, collectionId, 'folder', 'collection')

for folder in folders:
    print "cleaning up folder " + folder["name"]
    offset = -1
    folderId = folder["_id"]
    
    while True:
        items = list(gc.listItem(folderId, limit=500, offset=offset + 1))
        for item in items:
            name = item["name"]
        
            if name.startswith('TCGA-'):
                patientFolderName = name[0:12]
                slideFolderName = name.split(".",1)[0]
            
                try:
                    patientFolder = gc.createFolder(folderId, patientFolderName, parentType='folder')
                except:
                    requestUrl = 'resource/search?q=%s&mode=prefix&types=%%5B%%22folder%%22%%5D&level=0&limit=10&offset=0' % patientFolderName
                    patientFolder = gc.getResource(requestUrl)["folder"][0]
                    pass

                try:
                    gc.createFolder(patientFolder["_id"], slideFolderName)
                    print "Creating slide folder " + folderName
                except:
                    pass
                
        if len(items) < 500: break
        offset += 500

cleaning up folder acc
cleaning up folder blca
